In [28]:
from pathlib import Path

import pandas as pd

In [75]:
labels = {
	'stations': {
		'SE': 'sureste',
		'NE': 'noreste',
		'CE': 'centro',
		'NO': 'noroeste',
		'SO': 'suroeste',
		'NO2': ['noroeste2', 'noroeste 2'],
		'NTE': 'norte',
		'NE2': ['noreste2', 'noreste 2'],
		'SE2': ['sureste2', 'sureste 2'],
		'SO2': ['suroeste2', 'suroeste 2'],
		'SUR': 'sur',
		'NTE2': ['norte2', 'norte 2'],
		'SE3': ['sureste3', 'sureste 3'],
		'NE3': ['noreste3', 'noreste 3'],
		'NOE3': ['noroeste3', 'noroeste 3']
	},
	'contaminants': {
		'PM10': 'Partículas menores a 10 micras',
		'PM2.5': 'Partículas menores a 2.5 micras',
		'O3': 'Ozono',
		'SO2': 'Dióxido de azufre',
		'NO2': 'Dióxido de nitrógeno',
		'CO': 'Monóxido de carbono',
		'NO': 'Monóxido de nitrógeno',
		'NOX': 'Óxidos de nitrógeno'
	}
}

additional_labels = {
	'parameters': {
		'TOUT': 'Temperatura',
		'RH': 'Humedad Relativa',
		'SR': 'Radiación Solar',
		'RAINF': 'Precipitación',
		'PRS': 'Presión Atmosférica',
		'WSR': 'Velocidad del Viento',
		'WDR': 'Dirección del Viento'
    }
}

In [30]:
# Dataset one
df_2020_2021_all_stations = pd.read_excel(
	Path("../data/raw/DATOS HISTÓRICOS 2020_2021_TODAS ESTACIONES.xlsx"),
	sheet_name=None
)

In [31]:
# Process dataset 1
frames = []
for name, frame in df_2020_2021_all_stations.items():
	if name == 'NOROESTE3':
		continue
	for code, codename in labels['stations'].items():
		frame_copy = frame.copy()
		if isinstance(codename, list):
			if any(name.upper() == cn.upper() for cn in codename):
				frame_copy['station_code'] = code
				frames.append(frame_copy)
		else:
			if name.upper() == codename.upper():
				frame_copy['station_code'] = code
				frames.append(frame_copy)


df_2020_2021_all_stations_processed = pd.concat(frames, ignore_index=True)

In [32]:
# Dataset two
df_2022_2023_all_stations = pd.read_excel(
	Path("../data/raw/DATOS HISTÓRICOS 2022_2023_TODAS ESTACIONES.xlsx"),
	sheet_name=None
)

In [33]:
# Process dataset 2
frames = []
for name, frame in df_2022_2023_all_stations.items():
	for code, codename in labels['stations'].items():
		frame_copy = frame.copy()
		if isinstance(codename, list):
			if any(name.upper() == cn.upper() for cn in codename):
				frame_copy['station_code'] = code
				frames.append(frame_copy)
		else:
			if name.upper() == codename.upper():
				frame_copy['station_code'] = code
				frames.append(frame_copy)


df_2022_2023_all_stations_processed = pd.concat(frames, ignore_index=True)

In [34]:
# Dataset three
df_2023_2024_all_stations = pd.read_excel(
	Path("../data/raw/DATOS HISTÓRICOS 2023_2024_TODAS ESTACIONES_ITESM.xlsx"),
	sheet_name='Param_horarios_Estaciones',
	header=None
)

In [68]:
# Process dataset 3
stations_map = labels['stations']

station_name_to_code = {}
for code, names in stations_map.items():
    if isinstance(names, list):
        for name in names:
            station_name_to_code[name.upper()] = code
    else:
        station_name_to_code[names.upper()] = code

stations_row = df_2023_2024_all_stations.iloc[0, 1:].astype(str).str.strip()
vars_row = df_2023_2024_all_stations.iloc[1, 1:].astype(str).str.strip()

full_body = df_2023_2024_all_stations.iloc[3:].reset_index(drop=True)
dates = pd.to_datetime(full_body.iloc[:, 0], errors="coerce", dayfirst=True)
body = full_body.iloc[:, 1:]


contaminants = list(labels["contaminants"].keys())

In [67]:
body

,0,1,2,3,4,5,6,7,8,9,...,230,231,232,233,234,235,236,237,238,239
0,2023-01-01 00:00:00,2.37,54.5,32.6,87.1,3,110,68,721.7,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-01-01 01:00:00,2.12,38.7,30.3,68.9,3,116,67.18,721.5,0,...,246,NaN,707.4,0,59,2.9,0,14.14,65.3,95
2,2023-01-01 02:00:00,2.05,38.7,28.8,67.4,3,117,75.12,721.1,0,...,239,NaN,707,0,59,3.2,0,14.11,60.9,90
3,2023-01-01 03:00:00,2.5,60.5,29.1,89.4,3,135,82.81,720.8,0,...,275,NaN,706.8,0,59,2.3,0,14.08,58.1,84
4,2023-01-01 04:00:00,1.94,42.3,25.7,67.7,NaN,132,59.56,720.7,0,...,NaN,NaN,706.8,0,58,1.8,0,12.94,61.4,87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13865,2024-07-31 19:00:00,0.67,NaN,4.3,7.5,26,88,NaN,721.7,0,...,117,13.73,707.5,0,46,2.8,0.003,29.87,13.1,63
13866,2024-07-31 20:00:00,0.66,2.9,4.5,7.4,24,94,NaN,721.8,0,...,90,13.05,707.9,0,48,2.7,0,28.93,11.8,41
13867,2024-07-31 21:00:00,0.65,2.7,3.9,6.6,24,83,NaN,722.1,0,...,90,12.72,708.4,0,50,3,0,28.31,11.6,41
13868,2024-07-31 22:00:00,0.65,2.8,3.9,6.7,23,95,NaN,722.5,0,...,104,13.63,709.1,0,58,3,0,27.25,15.3,35


In [79]:
frames = []
for station in stations_row.unique():
    if pd.isna(station) or station == "nan" or station.upper() not in station_name_to_code:
        continue

    # Find which columns contain data for the current station
    station_columns = stations_row[stations_row == station].index.tolist()

    station_data = {
        "station_code": station_name_to_code[station.upper()],
        "date": dates
    }

    for col_idx in station_columns:
        if col_idx not in vars_row.index or col_idx not in body.columns:
            print(f"Warning: Column label {col_idx} not found in vars_row or body.")
            continue

        var_name = vars_row.loc[col_idx]
        # Normalize known aliases in dataset 3 (only)
        if var_name == "WDV":
            var_name = "WDR"

        if var_name in contaminants or var_name in additional_labels["parameters"]:
            column_data = body.loc[:, col_idx]
            station_data[var_name] = pd.to_numeric(column_data, errors='coerce')

    if len(station_data) > 3:
        station_df = pd.DataFrame(station_data)
        frames.append(station_df)


# Concatenate all station dataframes
df_2023_2024_all_stations_processed = pd.concat(frames, ignore_index=True)

In [80]:
# Show SE station on date 2024-01-01
df_2023_2024_all_stations_processed

,station_code,date,CO,NO,NO2,NOX,O3,PM10,PM2.5,PRS,RAINF,RH,SO2,SR,TOUT,WSR,WDR
0,SE,2023-01-01 00:00:00,2.37,54.5,32.6,87.1,3.0,110.0,68.00,721.7,0.0,68.0,3.5,0.000,16.39,3.2,257.0
1,SE,2023-01-01 01:00:00,2.12,38.7,30.3,68.9,3.0,116.0,67.18,721.5,0.0,72.0,3.4,0.000,15.17,3.3,278.0
2,SE,2023-01-01 02:00:00,2.05,38.7,28.8,67.4,3.0,117.0,75.12,721.1,0.0,71.0,3.6,0.000,14.82,3.7,278.0
3,SE,2023-01-01 03:00:00,2.50,60.5,29.1,89.4,3.0,135.0,82.81,720.8,0.0,68.0,3.8,0.000,15.51,3.6,197.0
4,SE,2023-01-01 04:00:00,1.94,42.3,25.7,67.7,NaN,132.0,59.56,720.7,0.0,73.0,3.6,0.000,13.81,4.9,271.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208045,NOE3,2024-07-31 19:00:00,1.18,4.2,11.1,15.2,16.0,117.0,13.73,707.5,0.0,46.0,2.8,0.003,29.87,13.1,63.0
208046,NOE3,2024-07-31 20:00:00,1.33,4.4,12.5,16.8,13.0,90.0,13.05,707.9,0.0,48.0,2.7,0.000,28.93,11.8,41.0
208047,NOE3,2024-07-31 21:00:00,1.47,4.6,12.1,16.6,12.0,90.0,12.72,708.4,0.0,50.0,3.0,0.000,28.31,11.6,41.0
208048,NOE3,2024-07-31 22:00:00,1.26,4.8,11.3,16.0,12.0,104.0,13.63,709.1,0.0,58.0,3.0,0.000,27.25,15.3,35.0


In [37]:
# Concat all dataframes
main_dataframe = pd.concat(
	[
		df_2020_2021_all_stations_processed,
		df_2022_2023_all_stations_processed,
		df_2023_2024_all_stations_processed
	],
	ignore_index=True
)

In [38]:
main_dataframe

,date,CO,NO,NO2,NOX,O3,PM10,PM2.5,PRS,RAINF,RH,SO2,SR,TOUT,WSR,WDR,station_code
0,2020-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,66.0,54.23,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,SE
1,2020-01-01 01:00:00,2.11,NaN,NaN,NaN,19.0,57.0,NaN,735.7,0.0,96.0,5.4,0.01,11.20,8.1,NaN,SE
2,2020-01-01 02:00:00,2.06,NaN,NaN,NaN,19.0,68.0,53.84,734.8,0.0,96.0,5.5,0.01,11.26,5.5,NaN,SE
3,2020-01-01 03:00:00,1.96,NaN,NaN,NaN,19.0,68.0,36.47,734.2,0.0,96.0,5.4,0.01,11.35,3.8,NaN,SE
4,2020-01-01 04:00:00,1.98,NaN,NaN,NaN,16.0,48.0,33.59,733.9,0.0,96.0,5.5,0.01,11.47,3.3,NaN,SE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
659356,2024-07-31 19:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NOE3
659357,2024-07-31 20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NOE3
659358,2024-07-31 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NOE3
659359,2024-07-31 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NOE3


In [60]:
Path("../data/processed").mkdir(parents=True, exist_ok=True)

main_dataframe.to_csv(
	Path("../data/processed/main_dataframe.csv"),
	index=False
)